# Time Series Cross Validation

In [1]:
import pandas as pd
import numpy as np

#we include some parallel CPU examples to speed up your CV!
#if you don't have joblib installed then uncomment the line below.
#!pip install joblib
from joblib import Parallel, delayed

#suppress ARIMA warnings
import warnings
warnings.filterwarnings('ignore')

Up till now we have used a single validation period to select our best model.  The weakness of that approach is that it gives you a sample size of 1 (that's better than nothing, but generally poor statistics!).  Time series cross validation is an approach to provide more data points when comparing models. In the classicial time series literature time series cross validation is called a **Rolling Forecast Origin**.  There may also be benefit of taking a **sliding window** approach to cross validaiton.  This second approach maintains a fixed sized training set.  I.e. it drops older values from the time series during validation.

## Rolling Forecast Origin

The following code and output provide a simplified view of how rolling forecast horizons work in practice.

In [2]:
def rolling_forecast_origin(train, min_train_size, horizon):
    '''
    Rolling forecast origin generator.
    '''
    for i in range(len(train) - min_train_size - horizon + 1):
        split_train = train[:min_train_size+i]
        split_val = train[min_train_size+i:min_train_size+i+horizon]
        yield split_train, split_val

In [3]:
full_series = [2502, 2414, 2800, 2143, 2708, 1900, 2333, 2222, 1234, 3456]

test = full_series[-2:]
train = full_series[:-2]
print('full training set: {0}'.format(train))
print('hidden test set: {0}'.format(test))

full training set: [2502, 2414, 2800, 2143, 2708, 1900, 2333, 2222]
hidden test set: [1234, 3456]


In [4]:
cv_rolling = rolling_forecast_origin(train, min_train_size=4, horizon=2)
cv_rolling

<generator object rolling_forecast_origin at 0x7f47c1cf2f48>

In [5]:
i = 0
for cv_train, cv_val in cv_rolling:
    print(f'CV[{i+1}]')
    print(f'Train:\t{cv_train}')
    print(f'Val:\t{cv_val}')
    print('-----')
    i += 1

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708, 1900]
-----
CV[2]
Train:	[2502, 2414, 2800, 2143, 2708]
Val:	[1900, 2333]
-----
CV[3]
Train:	[2502, 2414, 2800, 2143, 2708, 1900]
Val:	[2333, 2222]
-----


## Sliding Window Cross Validation

In [6]:
def sliding_window(train, window_size, horizon, step=1):
    '''
    sliding window  generator.
    
    Parameters:
    --------
    train: array-like
        training data for time series method
    
    window_size: int
        lookback - how much data to include.
    
    horizon: int
        forecast horizon
        
    step: int, optional (default=1)
        step=1 means that a single additional data point is added to the time
        series.  increase step to run less splits.
        
    Returns:
        array-like, array-like
    
        split_training, split_validation
    '''
    for i in range(0, len(train) - window_size - horizon + 1, step):
        split_train = train[i:window_size+i]
        split_val = train[i+window_size:window_size+i+horizon]
        yield split_train, split_val

This code tests its with `step=1`

In [7]:
cv_sliding = sliding_window(train, window_size=4, horizon=1)

print('full training set: {0}\n'.format(train))

i = 0
for cv_train, cv_val in cv_sliding:
    print(f'CV[{i+1}]')
    print(f'Train:\t{cv_train}')
    print(f'Val:\t{cv_val}')
    print('-----')
    i += 1

full training set: [2502, 2414, 2800, 2143, 2708, 1900, 2333, 2222]

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708]
-----
CV[2]
Train:	[2414, 2800, 2143, 2708]
Val:	[1900]
-----
CV[3]
Train:	[2800, 2143, 2708, 1900]
Val:	[2333]
-----
CV[4]
Train:	[2143, 2708, 1900, 2333]
Val:	[2222]
-----


The following code tests it with `step=2`.  Note that you get less splits.  The code is less computationally expensive at the cost of less data.  That is probably okay.

In [8]:
cv_sliding = sliding_window(train, window_size=4, horizon=1, step=2)

print('full training set: {0}\n'.format(train))

i = 0
for cv_train, cv_val in cv_sliding:
    print(f'CV[{i+1}]')
    print(f'Train:\t{cv_train}')
    print(f'Val:\t{cv_val}')
    print('-----')
    i += 1

full training set: [2502, 2414, 2800, 2143, 2708, 1900, 2333, 2222]

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708]
-----
CV[2]
Train:	[2800, 2143, 2708, 1900]
Val:	[2333]
-----


# Parallel Cross Validation Example using Naive1

In [9]:
from forecast_tools.baseline import SNaive, Naive1
from forecast_tools.datasets import load_emergency_dept
from sklearn.metrics import mean_absolute_error

In [10]:
train = load_emergency_dept()

In [11]:
def forecast_accuracy(model, train, test, metric):
    '''
    Returns forecast accuracy of model fit on 
    training data and compared against a test set 
    with a given metric.
    
    Parameters:
    ----------
    model - object
        forecasting model with .fit(train) and
        .predict(horizon) methods
        
    train - array-like
        training data
        
    test: array-like
        holdout data for testing
        
    metric: function
        error measure sig (y_true, y_preds)
    '''
    model.fit(train)
    preds = model.predict(len(test))
    return metric(y_true=test, y_pred=preds)

In [12]:
def cross_validation_score(model, train, cv, metric, n_jobs=-1):
    '''
    Calculate cross validation scores
    
    Parameters:
    ----------
    model: object
        forecast model
    
    train: array-like
        training data
        
    metric: func(y_true, y_pred)
        forecast error metric
    
    n_jobs: int, optional (default=-1)
        when -1 runs across all cores
        set = 1 to run each cross validation seperately.
        using -1 speeds up cross validation of slow running models.    
    '''
    cv_scores = Parallel(n_jobs=n_jobs)(delayed(forecast_accuracy)(model, 
                                                cv_train, 
                                                cv_test, 
                                                mean_absolute_error) 
                              for cv_train, cv_test in cv)

    return np.array(cv_scores)
    

In [13]:
model = Naive1()

In [14]:
#%%timeit runs the code multiple times to get an estimate of runtime.
#comment if out to run the code only once.

Run on a single core

In [15]:
#%%timeit
cv = sliding_window(train, window_size=14, horizon=7, step=1)
results_1 = cross_validation_score(model, train, cv, mean_absolute_error, 
                                   n_jobs=1)

Run across multiple cores by setting `n_jobs=-1`

In [16]:
#%%timeit #should see a little bit of speed up, but not loads with basic model.
cv = sliding_window(train, window_size=14, horizon=7, step=1)
results_2 = cross_validation_score(model, train, cv, mean_absolute_error,
                                       n_jobs=-1)

In [17]:
results_1.shape

(324,)

In [18]:
results_2.shape

(324,)

In [19]:
print(results_1.mean(), results_1.std())

26.653439153439148 10.346957901088238


just to illustrate that the results are the same - the difference is runtime.

In [20]:
print(results_2.mean(), results_2.std())

26.653439153439148 10.346957901088238


## Cross validation example using ARIMA - does it speed up when CV run in Parallel?

In [21]:
#use ARIMA from pmdarima as that has a similar interface to baseline models.
from pmdarima import ARIMA, auto_arima

In [22]:
#ato_model = auto_arima(train, suppress_warnings=True, n_jobs=-1, m=7)

In [23]:
#auto_model

In [27]:
#create arima model - reasonably complex model
#order=(1, 1, 2), seasonal_order=(2, 0, 2, 7)
args = {'order':(1, 1, 2), 'seasonal_order':(2, 0, 2, 7)}
model = ARIMA(order=args['order'], seasonal_order=args['seasonal_order'],
              enforce_stationarity=False, suppress_warnings=True)

In [30]:
%%time
cv = rolling_forecast_origin(train, min_train_size=320, horizon=7)
results_1 = cross_validation_score(model, train, cv, mean_absolute_error, 
                                   n_jobs=1)

CPU times: user 37.9 s, sys: 300 ms, total: 38.2 s
Wall time: 12.5 s


comment out %%timeit to run the code only once!

you should see a big improvement in performance.  mine went 
from 12.3 seconds to 2.4 seconds.

In [31]:
%%time
cv = rolling_forecast_origin(train, min_train_size=320, horizon=7)
results_2 = cross_validation_score(model, train, cv, mean_absolute_error, 
                                   n_jobs=-1)

CPU times: user 517 ms, sys: 292 ms, total: 809 ms
Wall time: 3.55 s


In [32]:
results_1.shape

(18,)

In [33]:
results_2.shape

(18,)

In [34]:
results_1.mean()

15.58663509219668

In [35]:
results_2.mean()

15.586662939018318